In [5]:
import pandas as pd
import numpy as np
import dateutil.parser
from datetime import date
import re

from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing


import pickle
import time

In [6]:
df=pd.read_csv('steam3.csv')
del(df['Unnamed: 0'])
df.head()

,title,link,system,systems,reviews,release_date,discount,price
0,Warframe,https://store.steampowered.com/app/230410/Warf...,1,['win'],192592,1940,0,0.00
1,Clicker Heroes 2,https://store.steampowered.com/app/629910/Clic...,2,"['win', 'mac']",183,1,0,29.99
2,Tom Clancy's Rainbow Six® Siege,https://store.steampowered.com/app/359550/Tom_...,1,['win'],170518,959,0,14.99
3,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...,3,"['win', 'mac', 'linux']",2628652,2156,0,14.99
4,Grand Theft Auto V,https://store.steampowered.com/app/271590/Gran...,1,['win'],343780,1191,34,29.99


In [7]:
no_dup_df = df[~df.index.duplicated(keep='first')]

In [8]:
no_dup_df['system'] = no_dup_df.system.astype(int)
no_dup_df['reviews'] = no_dup_df.reviews.astype(int)
no_dup_df['release_date'] = no_dup_df.release_date.astype(int)
no_dup_df['discount'] = no_dup_df.discount.astype(int)
no_dup_df['price'] = no_dup_df.price.astype(float)
no_dup_df['systems'] = no_dup_df.systems.astype(list)

# First train/test 10-fold split

In [18]:
y = df['reviews']
X = df.drop(['reviews','link','systems','title'], axis = 1)

In [19]:
kf = KFold(n=len(X), n_folds=10, shuffle=True)

# First K-fold

In [20]:
alphas = [10**x for x in range(-12, 5)]
scores = np.zeros((len(alphas),1))

In [21]:
for j in range(len(alphas)):
    kf_score = []
    for train, test in kf:
        X_train = X.iloc[train]
        y_train = y.iloc[train]
        X_test = X.iloc[test]
        y_test = y.iloc[test]
        std_scaler = preprocessing.StandardScaler()
        X_train_norm = std_scaler.fit_transform(X_train)
        X_test_norm = std_scaler.transform(X_test)
    
        model =  Lasso(alpha = alphas[j])
        model.fit(X_train_norm, y_train)
        y_test_predict = model.predict(X_test_norm)
        score = mean_squared_error(y_test_predict, y_test)
        kf_score.append(score)
    scores[j] = np.mean(kf_score)
print ('mse for all alpha values \n')
print (pd.DataFrame(list(zip(alphas, scores)), columns=['alpha', 'mse']))

/Users/mayamadhavan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/mayamadhavan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/mayamadhavan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/mayamadhavan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceW

mse for all alpha values 

           alpha                   mse
0   1.000000e-12  [1669091043.7629926]
1   1.000000e-11  [1669091043.7629926]
2   1.000000e-10  [1669091043.7629924]
3   1.000000e-09   [1669091043.762992]
4   1.000000e-08  [1669091043.7629774]
5   1.000000e-07  [1669091043.7628953]
6   1.000000e-06  [1669091043.7618766]
7   1.000000e-05  [1669091043.7534535]
8   1.000000e-04   [1669091043.660386]
9   1.000000e-03  [1669091042.3780773]
10  1.000000e-02  [1669091034.0285141]
11  1.000000e-01  [1669090949.8373675]
12  1.000000e+00  [1669090111.4988847]
13  1.000000e+01  [1669082085.4124963]
14  1.000000e+02  [1669047999.0449595]
15  1.000000e+03  [1671688079.7280502]
16  1.000000e+04   [1674688480.397724]


In [26]:
best_alpha = alphas[scores.argmin()]
print ('alpha:', best_alpha)

alpha: 100


In [27]:
std_scaler = preprocessing.StandardScaler()
X_norm = std_scaler.fit_transform(X)

final_model =  Lasso(alpha = best_alpha)
final_model.fit(X_norm, y)
y_predict = final_model.predict(X_norm)
final_model_score = mean_squared_error(y_predict, y)
print ('MSE:', final_model_score)

MSE: 1665513457.6148596


In [28]:
df_coef = pd.DataFrame(list(zip(X.columns, final_model.coef_)), columns = ['variable', 'coefficient'])
print ('Keep')
print (df_coef[df_coef['coefficient']!=0])

Keep
       variable  coefficient
0        system  1210.255576
1  release_date  2079.367651
3         price  1155.444291


In [29]:
print ('Drop')
print (df_coef[df_coef['coefficient']==0])

Drop
   variable  coefficient
2  discount          0.0


# Feature Eng/Refining Model

Unsure of Order
1. Run patsy on train/test
2. Diagnostic Plots
    + residuals etc...
3. Correlations/pairplot/heatmap
4. Compare predictions with other
5. Change features
6. (?????) Redo Lasso
7. Drop features
8. Box-cox?

In [30]:
y_transformed, _ = stats.boxcox()

NameError: name 'stats' is not defined